In [1]:
%load_ext autoreload
%autoreload

In [2]:
from kedro.runner.sequential_runner import SequentialRunner
from kedro.io.data_catalog import DataCatalog
from jaffle_shop.utilities.graph import kedro_node_to_graph, kedro_pipeline_to_graph, render_nx_graph

In [3]:
default_pipeline = pipelines['__default__']
nx_pipeline_graph = kedro_pipeline_to_graph(default_pipeline, catalog)
render_nx_graph(nx_pipeline_graph).show('graph.html')

graph.html


<IPython.lib.display.IFrame object at 0x173af6740>

In [4]:
n = default_pipeline.nodes[2]
nx_node_graph = kedro_node_to_graph(n,catalog)
render_nx_graph(nx_node_graph).show('graph.html')

graph.html


<IPython.lib.display.IFrame object at 0x177b7e440>

In [12]:
import networkx as nx

def get_lineage_for_kedro_node(pipe, node, data_catalog):
    
    def _retrieve_node_by_desc(g, key, value):
        for i, n in g.nodes(data=True):
            if n.get(key) == value:
                return i
    
    pipe = pipelines['__default__']
    pipeline_graph = kedro_pipeline_to_graph(pipe, catalog)
    node_graph = kedro_node_to_graph(node,catalog)
    combined_graphs = nx.disjoint_union(node_graph, pipeline_graph)
    
    for i in node.inputs:
        kedro_edge = _retrieve_node_by_desc(combined_graphs, "label", i)
        ibis_edge = _retrieve_node_by_desc(combined_graphs, "description", i)
        combined_graphs.add_edge(kedro_edge, ibis_edge)
        
    for o in node.outputs:
        kedro_edge = _retrieve_node_by_desc(combined_graphs, "label", o)
        ibis_edge = _retrieve_node_by_desc(combined_graphs, "description", o)
        combined_graphs.add_edge(ibis_edge, kedro_edge)
    
    return combined_graphs
    
    
render_nx_graph(
    get_lineage_for_kedro_node(
        default_pipeline,
        default_pipeline.nodes[2],
        catalog
    )
).show('graph.html')

graph.html


<IPython.lib.display.IFrame object at 0x288b29930>